In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class VGGBlock(nn.Module):
    def __init__(self, input_channel, output_channel, num_conv):
        super().__init__()
        module = []
        for _ in range(num_conv):
            module.append(nn.Conv2d(input_channel, output_channel, kernel_size=3, stride=1, padding=1))
            module.append(nn.ReLU(inplace=True))
            input_channel = output_channel

        self.conv_block = nn.Sequential(*module)

    def forward(self, x):
        x = self.conv_block(x)
        return x

In [3]:
class VGG16(nn.Module):
    def __init__(self, input_channel=3, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            # N x 3 x 224 x 224
            VGGBlock(input_channel, 64, 2),
            nn.MaxPool2d(2, stride=2),
            # N x 64 x 112 x 112
            VGGBlock(64, 128, 2),
            nn.MaxPool2d(2, stride=2),
            # N x 128 x 56 x 56
            VGGBlock(128, 256, 3),
            nn.MaxPool2d(2, stride=2),
            # N x 256 x 28 x 28
            VGGBlock(256, 512, 3),
            nn.MaxPool2d(2, stride=2),
            # N x 512 x 14 x 14
            VGGBlock(512, 512, 3),
            nn.MaxPool2d(2, stride=2),
            # N x 512 x 7 x 7
        )
        self.classifier = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1 ,512*7*7)
        x = self.classifier(x)
        return x

In [4]:
x = torch.randn(1, 3, 224, 224)
model = VGG16()
model(x)

tensor([[ 0.0193, -0.0107, -0.0095, -0.0065,  0.0059,  0.0066, -0.0002, -0.0052,
          0.0093, -0.0066]], grad_fn=<AddmmBackward0>)